In [62]:
# import statements cell
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
import sagemaker as sage
import numpy as np
import boto3
import io
import sagemaker.amazon.common as smac
import os
from sagemaker.amazon.amazon_estimator import get_image_uri
import mxnet as mx
from sagemaker.predictor import csv_serializer, json_deserializer
from io import StringIO
import json
from sagemaker import get_execution_role
from sagemaker.predictor import json_serializer, json_deserializer

In [11]:
# scrape raw CSV from NYTimes COVID-19 GitHub repo, parse with BeautifulSoup
response = requests.get('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')

text = StringIO(response.text)
raw_data = pd.read_csv(text)

In [12]:
# remove datapoints without FIPS code
# this includes New York City, that's a problem

trimmed_fips = raw_data[np.isfinite(raw_data['fips'])].copy(deep=True)
trimmed_fips['fips'] = trimmed_fips['fips'].astype(int)
trimmed_fips['cases'] = trimmed_fips['cases'].astype(int)
trimmed_fips['deaths'] = trimmed_fips['deaths'].astype(int)
trimmed_fips['date'] = trimmed_fips['date'].map(lambda date: datetime.strptime(date, '%Y-%m-%d').timestamp())
trimmed_fips = trimmed_fips.drop(columns=['county', 'state'])

In [13]:
# upload trimmed data from USDA
education = pd.read_excel('Education.xls')
# filter out states and countries
education = education[education['FIPS Code'] % 1000 != 0]

unemployment = pd.read_excel('Unemployment.xls')
# filter out states and countries
unemployment = unemployment[unemployment['FIPS'] % 1000 != 0]

poverty = pd.read_excel('PovertyEstimates.xls')
# filter out states and countries
poverty = poverty[poverty['FIPS'] % 1000 != 0]

population = pd.read_excel('PopulationEstimates.xls')
# filter out states and countries
population = population[population['FIPS'] % 1000 != 0]

atlas_people = pd.read_excel('RuralAtlasData20.xlsx', sheet_name='People')
# filter out states and countries
atlas_people = atlas_people[atlas_people['FIPS'] % 1000 != 0]

atlas_jobs = pd.read_excel('RuralAtlasData20.xlsx', sheet_name='Jobs')
# filter out states and countries
atlas_jobs = atlas_jobs[atlas_jobs['FIPS'] % 1000 != 0]

atlas_county_classifications = pd.read_excel('RuralAtlasData20.xlsx', sheet_name='County Classifications')
# filter out states and countries
atlas_county = atlas_county_classifications[atlas_county_classifications['FIPS'] % 1000 != 0]

atlas_income = pd.read_excel('RuralAtlasData20.xlsx', sheet_name='Income')
# filter out states and countries
atlas_income = atlas_income[atlas_income['FIPS'] % 1000 != 0]

atlas_veterans = pd.read_excel('RuralAtlasData20.xlsx', sheet_name='Veterans')
# filter out states and countries
atlas_veterans = atlas_veterans[atlas_veterans['FIPS'] % 1000 != 0]

In [14]:
# join socioeconomic measures by county together with COVID-19 spread data

data_with_edu = pd.merge(trimmed_fips, education, how='left', left_on='fips', right_on='FIPS Code')
data_with_edu_unemploy = pd.merge(data_with_edu, unemployment, how='left', left_on='fips', right_on='FIPS')
data_with_edu_unemploy_pov = pd.merge(
    data_with_edu_unemploy, poverty, how='left', left_on='fips', right_on='FIPS')
data_after_pop = pd.merge(
    data_with_edu_unemploy_pov, population, how='left', left_on='fips', right_on='FIPS')
data_after_atlas_people = pd.merge(
    data_after_pop, atlas_people, how='left', left_on='fips', right_on='FIPS')
data_after_atlas_jobs = pd.merge(
    data_after_atlas_people, atlas_jobs, how='left', left_on='fips', right_on='FIPS')
data_after_atlas_classif = pd.merge(
    data_after_atlas_jobs, atlas_county_classifications, how='left', left_on='fips', right_on='FIPS')
data_after_atlas_income = pd.merge(
    data_after_atlas_classif, atlas_income, how='left', left_on='fips', right_on='FIPS')
data_after_atlas_veterans = pd.merge(
    data_after_atlas_income, atlas_veterans, how='left', left_on='fips', right_on='FIPS')
data_with_usda = data_after_atlas_veterans
data_with_usda = data_with_usda.drop(columns=['FIPS Code', 'FIPS_x', 'FIPS_y'])

In [15]:
# we have some NaNs, SageMaker doesn't like these
# impute by mean of each column 
# from https://stackoverflow.com/questions/18689235/numpy-array-replace-nan-values-with-average-of-columns

def impute_by_means(a):
    col_means = np.nanmean(a, axis=0)
    inds = np.where(np.isnan(a))
    a[inds] = np.take(col_means, inds[1])
    return a

In [16]:
# features don't have cases or deaths

train, test = train_test_split(data_with_usda)

train_labels_cases = train['cases'].to_numpy(dtype=np.float32)
train_labels_deaths = train['deaths'].to_numpy(dtype=np.float32)
train_features = impute_by_means(train.drop(columns=['cases', 'deaths']).to_numpy(dtype=np.float32))

test_labels_cases = test['cases'].to_numpy(dtype=np.float32)
test_labels_deaths = test['deaths'].to_numpy(dtype=np.float32)
test_features = impute_by_means(test.drop(columns=['cases', 'deaths']).to_numpy(dtype=np.float32))

In [23]:
role = get_execution_role()
bucket = 'sagemaker-studio-uok86wzhfvl'
prefix = 'covid-data'

In [24]:
# from https://www.bmc.com/blogs/aws-linear-learner/
# and https://www.xaxis.com/insights/blog/steps-to-train-a-machine-learning-model-with-amazon-sagemaker-first-look/

sess = sage.Session(default_bucket=bucket)

In [25]:
key = 'fm'
s3_train_data_cases = 's3://{}/{}/train_cases/{}'.format(bucket, prefix, key)
s3_test_data_cases = 's3://{}/{}/test_cases/{}'.format(bucket, prefix, key)

s3_train_data_deaths = 's3://{}/{}/train_deaths/{}'.format(bucket, prefix, key)
s3_test_data_deaths = 's3://{}/{}/test_deaths/{}'.format(bucket, prefix, key)

In [26]:
# don't execute again unless data changed
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, train_features, train_labels_cases)
buf.seek(0)
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train_cases', key)).upload_fileobj(buf)

In [27]:
# don't execute again unless data changed
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, train_features, train_labels_deaths)
buf.seek(0)
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train_deaths', key)).upload_fileobj(buf)

In [28]:
# don't execute again unless data changed
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, test_features, test_labels_cases)
buf.seek(0)
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test_cases', key)).upload_fileobj(buf)

In [29]:
# don't execute again unless data changed
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, test_features, test_labels_deaths)
buf.seek(0)
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test_deaths', key)).upload_fileobj(buf)

In [30]:
output_loc_cases = 's3://{}/{}/output_cases'.format(bucket, prefix)
output_loc_deaths = 's3://{}/{}/output_deaths'.format(bucket, prefix)

In [31]:
container = get_image_uri(boto3.Session().region_name, 'factorization-machines')

In [34]:
estimator_cases = sage.estimator.Estimator(container, role, train_instance_count=1,
                                         train_instance_type='ml.c4.xlarge', output_path=output_loc_cases, 
                                         sagemaker_session=sess)
estimator_cases.set_hyperparameters(feature_dim=train_features.shape[1],
                        predictor_type='regressor',
                        num_factors=20)

In [35]:
estimator_deaths = sage.estimator.Estimator(container, role, train_instance_count=1,
                                         train_instance_type='ml.c4.xlarge', output_path=output_loc_deaths, 
                                         sagemaker_session=sess)
estimator_deaths.set_hyperparameters(feature_dim=train_features.shape[1],
                        predictor_type='regressor',
                        num_factors=20)

In [36]:
# run training job for cases
# don't run if not needed

estimator_cases.fit({'train': s3_train_data_cases, 'validation': s3_test_data_cases}, wait=False)
training_job_name_cases = estimator_cases.latest_training_job.job_name

In [37]:
# run training job for deaths
# don't run if not needed

estimator_deaths.fit({'train': s3_train_data_deaths, 'validation': s3_test_data_deaths}, wait=False)
training_job_name_deaths = estimator_deaths.latest_training_job.job_name

In [38]:
def get_training_job_status(training_job_name: str):
    job_info = boto3.client('sagemaker').describe_training_job(TrainingJobName=training_job_name)
    job_status = job_info['TrainingJobStatus']
    if job_status == 'Failed':
        message = job_info['FailureReason']
        print(f'Training failed with the following error: {message}')
    return job_status, job_info

In [41]:
get_training_job_status(estimator_deaths.latest_training_job.job_name)

('Completed',
 {'TrainingJobName': 'factorization-machines-2020-04-05-12-49-51-846',
  'TrainingJobArn': 'arn:aws:sagemaker:us-east-2:022575370123:training-job/factorization-machines-2020-04-05-12-49-51-846',
  'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-studio-uok86wzhfvl/covid-data/output_deaths/factorization-machines-2020-04-05-12-49-51-846/output/model.tar.gz'},
  'TrainingJobStatus': 'Completed',
  'SecondaryStatus': 'Completed',
  'HyperParameters': {'feature_dim': '502',
   'num_factors': '20',
   'predictor_type': 'regressor'},
  'AlgorithmSpecification': {'TrainingImage': '404615174143.dkr.ecr.us-east-2.amazonaws.com/factorization-machines:1',
   'TrainingInputMode': 'File',
   'MetricDefinitions': [{'Name': 'train:rmse:epoch',
     'Regex': '#quality_metric: host=\\S+, epoch=\\S+, train rmse <loss>=(\\S+)'},
    {'Name': 'train:progress',
     'Regex': '#progress_metric: host=\\S+, completed (\\S+) %'},
    {'Name': 'test:binary_f_beta',
     'Regex': '#quality_met

In [42]:
fm_predictor_cases = estimator_cases.deploy(initial_instance_count=1, wait=False, 
                                   instance_type='ml.c4.xlarge', endpoint_name='fm-covid19-cases')

In [63]:
fm_predictor_cases.content_type = 'application/json'
fm_predictor_cases.serializer = json_serializer
fm_predictor_cases.deserializer = json_deserializer

In [43]:
fm_predictor_deaths = estimator_deaths.deploy(initial_instance_count=1, wait=False,
                                   instance_type='ml.c4.xlarge', endpoint_name='fm-covid19-deaths')
fm_predictor_deaths.content_type = 'application/json'
fm_predictor_deaths.serializer = csv_serializer
fm_predictor_deaths.deserializer = json_deserializer

In [44]:
# join socioeconomic measures by county together

data_with_edu_unemploy = pd.merge(education, unemployment, how='left', left_on='FIPS Code', right_on='FIPS')
data_with_edu_unemploy_pov = pd.merge(
    data_with_edu_unemploy, poverty, how='left', left_on='FIPS', right_on='FIPS')
data_after_pop = pd.merge(
    data_with_edu_unemploy_pov, population, how='left', left_on='FIPS', right_on='FIPS')
data_after_atlas_people = pd.merge(
    data_after_pop, atlas_people, how='left', left_on='FIPS', right_on='FIPS')
data_after_atlas_jobs = pd.merge(
    data_after_atlas_people, atlas_jobs, how='left', left_on='FIPS', right_on='FIPS')
data_after_atlas_classif = pd.merge(
    data_after_atlas_jobs, atlas_county_classifications, how='left', left_on='FIPS', right_on='FIPS')
data_after_atlas_income = pd.merge(
    data_after_atlas_classif, atlas_income, how='left', left_on='FIPS', right_on='FIPS')
data_after_atlas_veterans = pd.merge(
    data_after_atlas_income, atlas_veterans, how='left', left_on='FIPS', right_on='FIPS')
county_data = data_after_atlas_veterans.drop(columns=['FIPS Code'])
county_data = county_data.set_index('FIPS')

In [45]:
client = boto3.client('sagemaker-runtime')
def linear_predictor(feature, label='cases'):
    csv_feature = StringIO('')
    df_temp = pd.DataFrame(feature)
    df_temp.transpose().to_csv(csv_feature, header=False, index=False)
    response = client.invoke_endpoint(EndpointName='predict-covid19-{}'.format(label),
                          Body=csv_feature.getvalue(), ContentType='text/csv')
    csv_feature.close()
    return json.load(response['Body'])

In [46]:
def xgboost_predictor(feature, label='cases'):
    csv_feature = StringIO('')
    df_temp = pd.DataFrame(feature)
    df_temp.transpose().to_csv(csv_feature, header=False, index=False)
    response = client.invoke_endpoint(EndpointName='xgboost-covid19-{}'.format(label),
                                     Body=csv_feature.getvalue(), ContentType='text/csv')
    csv_feature.close()
    return json.load(response['Body'])

In [75]:
def fm_predictor(feature, label='cases'):
    data = {'instances': [{'features': np.asarray(feature).astype(float).tolist()}]}
    response = client.invoke_endpoint(EndpointName='fm-covid19-{}'.format(label),
                          Body=json.dumps(data), ContentType='application/json')
    return json.load(response['Body'])

In [49]:
# a feature should have: date (Unix format), fips number, county data
def get_linear_prediction(date, fips, label='cases'):
    a = np.array([date, fips], dtype=np.float32)
    b = county_data.loc[fips].to_numpy(np.float32)
    feature = np.concatenate((a, b))
    return linear_predictor(feature, label)

In [50]:
# a feature should have: date (Unix format), fips number, county data
def get_xgboost_prediction(date, fips, label='cases'):
    a = np.array([date, fips], dtype=np.float32)
    b = county_data.loc[fips].to_numpy(np.float32)
    feature = np.concatenate((a, b))
    return xgboost_predictor(feature, label)

In [76]:
# a feature should have: date (Unix format), fips number, county data
def get_fm_prediction(date, fips, label='cases'):
    a = np.array([date, fips], dtype=np.float32)
    b = county_data.loc[fips].to_numpy(np.float32)
    feature = np.concatenate((a, b))
    return fm_predictor(feature, label)

In [51]:
get_linear_prediction((datetime.now() + timedelta(0)).timestamp(), 39103, label='cases')

{'predictions': [{'score': 166.59375}]}

In [52]:
get_xgboost_prediction((datetime.now() + timedelta(-10)).timestamp(), 39103, label='cases')

19.50457000732422

In [113]:
# fm predictions are ridiculously off

get_fm_prediction((datetime.now() + timedelta(-10)).timestamp(), 39103, label='deaths')

{'predictions': [{'score': -201273868288.0}]}